In [1]:
from pyomo.dae import ContinuousSet, DerivativeVar
from pyomo.environ import ConcreteModel, TransformationFactory, Var, \
                          NonNegativeReals, Constraint, \
                          SolverFactory, Objective, cos, sin, minimize, \
                          NonNegativeReals
import numpy as np
import matplotlib.pyplot as plt

# Define parameters of the problem
g=9.81
m_e=1e2
theta=10
C_RR=0.0035
C_D=0.65
rho=1.23
V=3 #m/s
U_max=16 #m/s, approx equivalent to 60km/h
AWC=1.5e4
CP=335
A=0.423
mm=m_e/1.014 #effective body mass 1.4% larger than actual mass
P_max=-((4*0.039*90+CP)/pow((0.01*90+163),2))*pow(90,2)+(4*(0.039*90+CP)/pow((0.01*90+163),2))*90 
#assumption only, equation (2.8) in Ashtiani 2021 thesis: 
#Optimal Pacing of Cyclists in a Times Trial based on Experimentally Calibrated Models of Fatigue and Recovery

model = ConcreteModel('CyclingRace')
model.T = Var(domain=NonNegativeReals)
model.t = ContinuousSet(bounds=(0, 1))
model.s = Var(model.t, domain=NonNegativeReals,bounds=(0,2e4))
model.U = Var(model.t, domain=NonNegativeReals,bounds=(1,20))
model.W = Var(model.t, domain=NonNegativeReals,bounds=(0,AWC))
model.P = Var(model.t, domain=NonNegativeReals,bounds=(0,P_max))
model.sdot = DerivativeVar(model.s, wrt=model.t, domain=NonNegativeReals)
model.Udot = DerivativeVar(model.U, wrt=model.t, domain=NonNegativeReals)
model.Wdot = DerivativeVar(model.W, wrt=model.t, domain=NonNegativeReals)

# Dynamics
model.sode = Constraint(model.t, rule=lambda m, t: m.sdot[t] == m.U[t]*m.T)


In [2]:
model.Uode=Constraint(model.t, rule=lambda m, t: m.Udot[t]==(m.P[t]/m.U[t]/m_e)-(mm/m_e) * g * (sin(theta)+C_RR * cos(theta))-0.5/m_e * C_D * A * (m.U[t]-V)**2*m.T)

In [3]:
model.Wode=Constraint(model.t, rule=lambda m, t: m.Wdot[t]== -(m.P[t]-CP)*m.T)

In [4]:
model.s[1].fix(2e4)

In [5]:
model.obj = Objective(expr=model.T, sense=minimize)
solver = SolverFactory('ipopt')
results = solver.solve(model)

KeyError: 2094599515104